In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
import struct
import torch.optim as optim
import torchvision
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import time
import random

In [2]:
src_vocab_size = 25002
d_model = 512
n_layers = 6
n_head = 8
d_k = 64
d_v = 64
d_ff = 2048
tgt_vocab_size = 25002
tgt_len = 25002
n_context = 8
d_context = 64

In [3]:
'''embeddings=torch.Tensor(25003,512)
embeddings.to("cuda")
with open('MetaAI04.vd','rb') as file:
    file.read(4)
    file.read(4)
    for i in range(0,25001):
        embeddings[i]=torch.Tensor(struct.unpack('<512f',file.read(4*512)))
'''

'embeddings=torch.Tensor(25003,512)\nembeddings.to("cuda")\nwith open(\'MetaAI04.vd\',\'rb\') as file:\n    file.read(4)\n    file.read(4)\n    for i in range(0,25001):\n        embeddings[i]=torch.Tensor(struct.unpack(\'<512f\',file.read(4*512)))\n'

In [4]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super().__init__()
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, unbiased=False, keepdim=True)

        out = (x - mean) / torch.sqrt(var + self.eps)
        #out = self.gamma * out + self.beta
        return out

In [5]:
def get_attn_pad_mask(seq_q, seq_k): 
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.data.eq(25001).unsqueeze(1)
    return pad_attn_mask.expand(batch_size, len_q, len_k)

In [6]:
def get_attn_subsequence_mask(seq):
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequence_mask = np.triu(np.ones(attn_shape), k=1)
    subsequence_mask = torch.from_numpy(subsequence_mask).byte()
    return subsequence_mask

In [7]:
class Embeddings(nn.Module):
    def __init__(self, vocab, d_model):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(src_vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [8]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=10000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].clone().detach()
        return self.dropout(x)

In [9]:
def get_sinusoid_encoding_table(max_len, d_model):
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe

In [10]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, scale_factor, dropout=0.1):
        super().__init__()
        self.scale_factor = scale_factor
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        attn = torch.matmul(q / self.scale_factor, k.transpose(2, 3))

        if mask is not None:
            attn = attn.masked_fill(mask == 1, -1e9)
        attn = self.dropout(torch.softmax(attn, dim=-1))
        output = torch.matmul(attn, v)
        return output, attn

In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_ks = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_vs = nn.Linear(d_model, n_head * d_v, bias=False)
        self.fc = nn.Linear(n_head * d_v, d_model, bias=False)

        self.attention = ScaledDotProductAttention(scale_factor=d_k ** 0.5)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)

    def forward(self, q, k, v, mask=None):
        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head
        batch_size, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)
        residual = q
        q = self.layer_norm(q)
        k = self.layer_norm(k)
        v = self.layer_norm(v)

        q = self.w_qs(q).view(batch_size, len_q, n_head, d_k)
        k = self.w_ks(k).view(batch_size, len_k, n_head, d_k)
        v = self.w_vs(v).view(batch_size, len_v, n_head, d_v)

        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1)
        q, attn = self.attention(q, k, v, mask=mask)

        q = q.transpose(1, 2).contiguous().view(batch_size, len_q, -1)
        q = self.dropout(self.fc(q))
        q += residual
        q = self.layer_norm(q)
        return q, attn

In [12]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, dropout=0.1):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False))
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        residual = inputs
        output = self.fc(inputs)
        output = self.dropout(output)
        return nn.LayerNorm(d_model).cuda()(output + residual)

In [13]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(n_head, d_model, d_k, d_v)
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, 
                                               enc_self_attn_mask)
        enc_outputs = self.pos_ffn(enc_outputs)
        return enc_outputs, attn

In [14]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.src_emb = Embeddings(src_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        enc_outputs = self.src_emb(enc_inputs)
        enc_outputs = self.pos_emb(enc_outputs)
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
        enc_self_attns = []
        for layer in self.layers:
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

In [15]:
class Decider(nn.Module):
    def __init__(self):
        super(Decider, self).__init__()
        
        self.dec_context_attn = MultiHeadAttention(n_head, d_model, d_k, d_v)
        self.dec_input_attn = MultiHeadAttention(n_head, d_model, d_k, d_v)
        self.context = torch.Tensor(n_context * d_context, d_model)
        self.contexts = torch.stack([self.context for _ in range(1)]).cuda()
    
    def clean(self, dim = 16):
        self.contexts = torch.stack([self.context for _ in range(dim)]).cuda()

    def forward(self, dec_inputs, enc_inputs):
        #self.contexts.requires_grad_(False)
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
        dec_outputs, attn = self.dec_input_attn(dec_inputs, self.contexts, self.contexts)
        contexts, attn = self.dec_context_attn(self.contexts, dec_inputs, dec_inputs, enc_self_attn_mask)
        return dec_outputs

In [16]:
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention(n_head, d_model, d_k, d_v)
        self.dec_enc_attn = MultiHeadAttention(n_head, d_model, d_k, d_v)
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, 
                                                 dec_inputs, dec_self_attn_mask)
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, 
                                                enc_outputs, dec_enc_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs)
        return dec_outputs, dec_self_attn, dec_enc_attn

In [17]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.tgt_emb = Embeddings(tgt_vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        dec_outputs = self.tgt_emb(dec_inputs)
        dec_outputs = self.pos_emb(dec_outputs)
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda()
        dec_self_attn_subsequent_mask = get_attn_subsequence_mask(dec_inputs).cuda()
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)

        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns

In [18]:
class MetaAI(nn.Module):
    def __init__(self):
        super(MetaAI, self).__init__()
        self.encoder = Encoder()
        self.decider = Decider()
        self.decoder = Decoder()
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False)

    def forward(self, enc_inputs, dec_inputs):
        dec_logits = []
        self.decider.clean(dim=1)
        for i in range(len(enc_inputs)):
            enc_outputs, enc_self_attns = self.encoder(enc_inputs[i].unsqueeze(0))
            #enc_outputs = self.decider(enc_outputs,enc_inputs[i].unsqueeze(0))
            dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs[i].unsqueeze(0), enc_inputs[i].unsqueeze(0), enc_outputs)
            dec_logits.append(self.projection(dec_outputs))
            dec_logits[i] = dec_logits[i].view(-1,dec_logits[i].size(-1))
        return dec_logits, enc_self_attns, dec_self_attns, dec_enc_attns

In [19]:
#torch.serialization.add_safe_globals([MetaAI, set, Encoder, Embeddings, PositionalEncoding, nn.Embedding, nn.Dropout, nn.ModuleList, EncoderLayer])
AI=MetaAI()
AI.load_state_dict(torch.load('MetaAI4.pth',weights_only = True),strict=False)
# for p in AI.parameters():
#     if p.dim() > 1:
#         nn.init.xavier_uniform_(p)
if torch.cuda.is_available():
    AI.to("cuda")

In [20]:
from datasets import load_dataset
dataset=load_dataset("chujiezheng/wizard_of_wikipedia",download_mode="reuse_dataset_if_exists")

In [21]:
def list_of_numbers_to_strings(lst):
    return [str(item) if isinstance(item, int) else list_of_numbers_to_strings(item) for item in lst]
def flatten_array(arr):
    return [item for sublist in arr for item in sublist]
def flatten_array_a(arr):
    return ['<|beginoftext|>'+item for sublist in arr for item in sublist]
def flatten_array_b(arr):
    return [item+'<|endoftext|>' for sublist in arr for item in sublist]
def _a(arr):
    for i in range(len(arr)):
        arr[i]='<|beginoftext|>'+arr[i]
    return arr
def _b(arr):
    for i in range(len(arr)):
        arr[i]=arr[i]+'<|endoftext|>'
    return arr

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my-new-tokenizer")
tokenizer.add_tokens(['<|beginoftext|>','<pad>'])
tokenizer.add_special_tokens({'pad_token': '<pad>'})
tokenizer.add_special_tokens({'bos_token': '<|beginoftext|>'})
tokenizer.add_special_tokens({'eos_token': '<|endoftext|>'})

batch = 1000

data_input_t=tokenizer(flatten_array(dataset["train"]['post'][:batch]),padding='longest')['input_ids']
data_output_t=tokenizer(flatten_array_a(dataset["train"]['response'][:batch]),padding='longest')['input_ids']
data_pred_t=tokenizer(flatten_array_b(dataset["train"]['response'][:batch]),padding='longest')['input_ids']

data_input=data_input_t
data_output=data_output_t
data_pred=data_pred_t
batchstart=[0]
fastreader = dataset["train"]['post'][:batch]
for i in range(1,batch):
    batchstart.append(len(fastreader[i]))
    batchstart[i]+=batchstart[i-1]

# for i in range(1000):
#     data_input.append([data_input_t[lst+_] for _ in range(len(dataset["train"]['post'][:1000][i]))])
#     lst+=len(dataset["train"]['post'][:1000][i])

# for i in range(len(dataset["train"]['post'][:1000])):
#     data_input.append(tokenizer(dataset["train"]['post'][:1000][i],padding='longest')['input_ids'])
#     data_output.append(tokenizer(_a(dataset["train"]['response'][:1000][i]),padding='longest')['input_ids'])
#     data_pred.append(tokenizer(_b(dataset["train"]['response'][:1000][i]),padding='longest')['input_ids'])

In [23]:
def get_batch(batch):
    return torch.tensor(data_input[batchstart[batch]:batchstart[batch]+len(fastreader[batch])]),torch.tensor(data_output[batchstart[batch]:batchstart[batch]+len(fastreader[batch])]),torch.tensor(data_pred[batchstart[batch]:batchstart[batch]+len(fastreader[batch])])

In [24]:
AI.eval()
a,b,c=get_batch(11)
output, un1, un2, un3 = AI([a[2].cuda()],[b[2].cuda()])
torch.argmax(output[0],dim=1)

tensor([   46,    79,   329,  9930,   812,   329,  5749,     7,    84,    27,
           84,   553,   290,   693,  4225,    12,   329,    27,    77,   473,
         1408,   403,   261, 18357,  2210,   281,  3560,  1462,   819,   401,
         2287,   490,   364,  2181,    14,     0,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667],
       device='cuda:0')

In [25]:
print("output: {} \n answer: {} \n correction: {}".format(torch.argmax(output[0],dim=1), c[2], (c[2].cuda()==25001)+(c[2].cuda()==torch.argmax(output[0],dim=1))))

output: tensor([   46,    79,   329,  9930,   812,   329,  5749,     7,    84,    27,
           84,   553,   290,   693,  4225,    12,   329,    27,    77,   473,
         1408,   403,   261, 18357,  2210,   281,  3560,  1462,   819,   401,
         2287,   490,   364,  2181,    14,     0,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,
         8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667,  8667],
       device='cuda:0') 
 answer: tensor([   46,    79,   329,

In [36]:
criterion = nn.CrossEntropyLoss(ignore_index=25001)
# optimizer = optim.Adam(AI.parameters(), lr=0.005, betas=(0.9, 0.98), eps=1e-9)
optimizer = optim.SGD(AI.parameters(), lr=0.016)

batchsize = 16
warmup_steps = 200

#inputd = inputd.view(-1,batchsize,inputd.size(1))
# inputd = torch.split(inputd.long(), batchsize, dim=0)
# outputd = torch.split(outputd.long(), batchsize, dim=0)
# pred = torch.split(pred.long(), batchsize, dim=0)
#for i in range(len(pred)):
#    pred[i] = F.one_hot(pred[i], num_classes=tgt_vocab_size).float()

if torch.cuda.is_available():
    criterion=criterion.cuda()

work_dir = 'C:/'
writer = SummaryWriter("{}/logs/{}".format(work_dir,str(random.randint(10000,99999))), comment=str(random.randint(10000,99999)))

In [37]:
last_epoch = 0
last_batch = 0

In [40]:
AI.train()

t0=time.time()

for epoch in range(last_epoch, 100):
    for i in range(last_batch,batch):
        optimizer.zero_grad()
        inputd,outputd,pred = get_batch(i)
        outputs, un1, un2, un3 = AI(inputd.cuda(), outputd.cuda())
        loss_total = 0
        for j in range(len(inputd)):
#             print(outputs[j].size())
#             print(pred[j].size())
            loss = criterion(outputs[j].contiguous().view(-1, tgt_vocab_size), pred[j].contiguous().view(-1).cuda())
            loss.backward()
            loss_total += loss.item()
#             print(loss.item())
        optimizer.step()
        train_step = batch*epoch+i+1
        #rate = d_model ** (-0.5) * min(train_step ** (-0.5), train_step * warmup_steps ** (-1.5))
        #for p in optimizer.param_groups:
        #    p['lr'] = rate
        if train_step % 20 == 0:
            print("train time：{}, Loss: {}".format(train_step, loss_total / len(inputd)))
        if train_step % 500 == 0:
            print("output: {} \n answer: {} \n correction: {}".format(torch.argmax(outputs[0],dim=1), pred[0].cuda(), (pred[0].cuda()==25001)+(pred[0].cuda()==torch.argmax(outputs[0],dim=1)) ))
        writer.add_scalar("train_loss", loss_total / len(inputd), train_step)
        last_batch=i
    print("time per epoch: {} minutes".format((time.time()-t0)/(epoch+1)/60))
    last_batch=0
    last_epoch=epoch

train time：320, Loss: 3.425010532140732
train time：340, Loss: 2.995445156097412
train time：360, Loss: 1.2482065558433533
train time：380, Loss: 1.7308690786361693
train time：400, Loss: 2.0347853899002075
train time：420, Loss: 2.513591468334198
train time：440, Loss: 2.675833895802498
train time：460, Loss: 2.6052913665771484
train time：480, Loss: 2.0045367181301117
train time：500, Loss: 1.7898331334193547
output: tensor([  55,  479,   12, 1643, 2716,    7,   84, 2255,  358,  358,   12, 1643,
        1643,  281, 1421,   12, 1643,  747,    1, 3322, 3914,  677,  434, 3914,
         286,  286,    1,    0,   70,  286,  286,  364,  261,  434,  271,  677,
         290,   83,  261,  358,  261, 3883, 8224,    1,    0,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1, 

train time：1520, Loss: 1.6414456764856975
train time：1540, Loss: 1.9752378344535828
train time：1560, Loss: 3.046063482761383
train time：1580, Loss: 1.2331477254629135
train time：1600, Loss: 1.6425396278500557
train time：1620, Loss: 2.6439135372638702
train time：1640, Loss: 2.0651017785072328
train time：1660, Loss: 2.4619945883750916
train time：1680, Loss: 1.6339953641096752
train time：1700, Loss: 3.420452296733856
train time：1720, Loss: 2.3279868960380554
train time：1740, Loss: 0.9197007775306701
train time：1760, Loss: 1.4801776558160782
train time：1780, Loss: 2.858782251675924
train time：1800, Loss: 2.7661605775356293
train time：1820, Loss: 2.679353803396225
train time：1840, Loss: 2.3312071204185485
train time：1860, Loss: 3.8055390119552612
train time：1880, Loss: 3.345185915629069
train time：1900, Loss: 2.8990938663482666
train time：1920, Loss: 2.6920234858989716
train time：1940, Loss: 3.5279040336608887
train time：1960, Loss: 2.5074410140514374
train time：1980, Loss: 1.41499919891357

train time：2800, Loss: 2.610824406147003
train time：2820, Loss: 2.5534353733062742
train time：2840, Loss: 2.1811409473419188
train time：2860, Loss: 3.612252414226532
train time：2880, Loss: 3.000482678413391
train time：2900, Loss: 2.5829732020696006
train time：2920, Loss: 2.431773394346237
train time：2940, Loss: 3.2104247212409973
train time：2960, Loss: 1.9994394481182098
train time：2980, Loss: 1.1707877576351167
train time：3000, Loss: 3.0588440895080566
output: tensor([  41,  769,   69,  364,  559,  281,  261,    7,   83,  693,  364, 4828,
         284,  329,  553,  358,  364, 3883,  358,  364, 6696,  364,  261, 1339,
          14,  221,   12,  290,  288,  290,   12,  290,  290,   12,  453,   12,
         290,  290,  290,  358,   12,  453,   14,   12,  290,   14,    1,    1,
         290,  290,  290,  290,  290,   12,  221,   12,  302,    1,  812,   12,
         358,  290,  463,   14,  290,  288,  290,  290,  358,   14,  290,  290,
          14,   14,   12,  358,   14,  290,   12,  302

train time：4020, Loss: 1.5191735494881868
train time：4040, Loss: 1.7588470876216888
train time：4060, Loss: 2.5214683413505554
train time：4080, Loss: 1.8416639417409897
train time：4100, Loss: 1.2198469787836075
train time：4120, Loss: 1.284528911113739
train time：4140, Loss: 2.0512676437695823
train time：4160, Loss: 2.3831084966659546
train time：4180, Loss: 1.5940485447645187
train time：4200, Loss: 0.8243550608555476
train time：4220, Loss: 1.3594954162836075
train time：4240, Loss: 1.4380584160486858
train time：4260, Loss: 1.4434857666492462
train time：4280, Loss: 1.2866633057594299
train time：4300, Loss: 1.4133170396089554
train time：4320, Loss: 2.618557393550873
train time：4340, Loss: 2.27355101108551
train time：4360, Loss: 0.7962979475657145
train time：4380, Loss: 0.974596482515335
train time：4400, Loss: 1.191584825515747
train time：4420, Loss: 1.57528904825449
train time：4440, Loss: 2.130998358130455
train time：4460, Loss: 1.9447204321622849
train time：4480, Loss: 1.1285261027514935
t

train time：5300, Loss: 1.2572564333677292
train time：5320, Loss: 2.4391315579414368
train time：5340, Loss: 2.1029442071914675
train time：5360, Loss: 0.8001591339707375
train time：5380, Loss: 0.8551692187786102
train time：5400, Loss: 1.0225048899650573
train time：5420, Loss: 1.5083471462130547
train time：5440, Loss: 1.8892849832773209
train time：5460, Loss: 1.7186316028237343
train time：5480, Loss: 1.0932536274194717
train time：5500, Loss: 1.0888535653551419
output: tensor([  55,  479,   12, 1643, 2857,    7,   84, 2255,  756,  358,  508, 1421,
         673,  855, 1421,   12, 1643,  747, 3428, 3322,  290,  677,  434, 9180,
        2478,  286,  364,    0,   70,  286,  286,  364, 9970,  677,  271,  677,
        1146,   83, 2491,  358, 5678, 3883, 2013,    1,    0,  473, 2255,  473,
         281,  281,  677,  281,  281,  281,  677,  473,  281,  281,  473, 2255,
        2255,  677, 3883,  812,  281, 2255, 2255,    1,  473,  473, 3883, 1408,
         473,  281,  281,  281,  281, 2255,  473, 

...

train time：25440, Loss: 0.26081401016563177
train time：25460, Loss: 0.3325518788769841
train time：25480, Loss: 0.08756757713854313
train time：25500, Loss: 0.19752181321382523
output: tensor([   55,   479,    12,   329,  2857,     7,    84,  2255,   756,   358,
          508,  1421,   673,  7787, 13379,    12,  1643,   747,  3428,  3322,
        11974, 21793,   434,  9180, 22294,   286,   364,  3370,    70, 22294,
          286,   364,  9970, 13328,   271,   677,  1146,    83,  2491,   358,
         8827,  3883,  2013,     1,     0,     1,   812,     1,   364,   709,
            1,   709,     1,   709,  1408,     1,  1408,     1,   394,   394,
         1408,     1,   434,     1,   434,     1,     1,   434,     1,  1408,
            1,     1,     1,     1,     1,     1,     1,  1421,   434,  1408,
          364,  1408,     1,     1,   364,     1,     1,     1,     1,   281,
          364,     1,  1408,  1408,   606,  1574,     1,   434,   434,     1,
          434,     1,     1,   364,  

train time：26520, Loss: 0.05798732861876488
train time：26540, Loss: 0.1407766687683761
train time：26560, Loss: 0.20435765199363232
train time：26580, Loss: 0.07890138542279601
train time：26600, Loss: 0.07464280258864164
train time：26620, Loss: 0.1601993814110756
train time：26640, Loss: 0.14882045965641738
train time：26660, Loss: 0.3132480134566625
train time：26680, Loss: 0.17736243704954782
train time：26700, Loss: 0.36645600013434887
train time：26720, Loss: 0.1444625398144126
train time：26740, Loss: 0.12787606874480845
train time：26760, Loss: 0.13152771443128586
train time：26780, Loss: 0.3959488794207573
train time：26800, Loss: 0.2705882843583822
train time：26820, Loss: 0.516626306436956
train time：26840, Loss: 0.19460764080286025
train time：26860, Loss: 0.5548914410173893
train time：26880, Loss: 0.39443185925483704
train time：26900, Loss: 0.3573960984746615
train time：26920, Loss: 0.34417247027158737
train time：26940, Loss: 0.33158006332814693
train time：26960, Loss: 0.0943971034139394

train time：27620, Loss: 0.17585923429578543
train time：27640, Loss: 0.1678120207041502
train time：27660, Loss: 0.25635263820489246
train time：27680, Loss: 0.1443654124935468
train time：27700, Loss: 0.3413214888423681
train time：27720, Loss: 0.1302690990269184
train time：27740, Loss: 0.09789410643279553
train time：27760, Loss: 0.20855445973575115
train time：27780, Loss: 0.3075302590926488
train time：27800, Loss: 0.2289806269109249
train time：27820, Loss: 0.4547509679570794
train time：27840, Loss: 0.14927096664905548
train time：27860, Loss: 0.5772981569170952
train time：27880, Loss: 0.3888806601365407
train time：27900, Loss: 0.314778375128905
train time：27920, Loss: 0.2775045596063137
train time：27940, Loss: 0.2869945038110018
train time：27960, Loss: 0.08899808302521706
train time：27980, Loss: 0.033781392872333525
train time：28000, Loss: 0.2557854223996401
output: tensor([  447,   769,    69,   364,   559,   281,  1822,     7,    83,   988,
         4753,  4828,    14,   329, 21581,   35

train time：28780, Loss: 0.2845877756675084
train time：28800, Loss: 0.19835292268544436
train time：28820, Loss: 0.45505195800215004
train time：28840, Loss: 0.16601889878511428
train time：28860, Loss: 0.4925236999988556
train time：28880, Loss: 0.3441740075747172
train time：28900, Loss: 0.3170030315717061
train time：28920, Loss: 0.25975281838327646
train time：28940, Loss: 0.2138468325138092
train time：28960, Loss: 0.05496000777930021
train time：28980, Loss: 0.030901755578815936
train time：29000, Loss: 0.17685313615947962
output: tensor([  447,   769,    69,   364,   559,   281,  1822,     7,    83,   988,
         4753,  4828,    14,   329, 21581,   358,   364,  1911,   358,   364,
         6696,  3690,   261,  1284,    14,     0,    12,   358,   281,   358,
           14,     0,    14,    14,   358,   281,     0,   358,   463,   463,
           14,  1156,   358,     0,   358,  1156,    14,   358,     0,   281,
          358,   281,  8905,   358,     0,   281,   281,   281,   302,   281,


KeyboardInterrupt: 

In [41]:
torch.save(AI.state_dict(),'MetaAI4.pth')

In [42]:
writer.close()